# Tutorial 1: Basics of CrypTen Tensors

We now have a high-level understanding of how secure MPC works. Through these tutorials, we will explain how to use CrypTen to carry out secure operations on encrypted tensors. In this tutorial, we will introduce a fundamental building block in CrypTen, called a ```CrypTensor```.  ```CrypTensors``` are encrypted ```torch``` tensors that can be used for computing securely on data. 

CrypTen currently only supports secure MPC protocols (though we intend to add support for other advanced encryption protocols). Using the ```mpc``` backend, ```CrypTensors``` act as ```torch``` tensors whose values are encrypted using secure MPC protocols. Tensors created using the ```mpc``` backend are called ```MPCTensors```. We will go into greater detail about ```MPCTensors``` in Tutorial 2. 

Let's begin by importing ```crypten``` and ```torch``` libraries. (If the imports fail, please see the installation instructions in the README.)

In [1]:
import crypten
import torch
from crypten.config import cfg
import timeit
import crypten.communicator as comm
from crypten.config import cfg
import logging

crypten.init()
logging.getLogger().setLevel(logging.INFO)


# comm.get().set_verbosity(True)
# ... do stuff ..
# comm.get().print_communication_stats()

/Users/memo/.pyenv/versions/3.11.4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import logging

# logging.basicConfig(filename='mylogs.log', level=logging.DEBUG)

# torch.set_printoptions(sci_mode=False)

#Construct example input CrypTensor
x = torch.arange(-10, 10, 0.1)
print(x.size())
# x = torch.tensor([0.01, 0.5])
x_enc = crypten.cryptensor(x)
num_iters = 1
print_terms = 8


# print("Exponential:")
# print("  Exact :", x.exp()[:print_terms])
# with cfg.temp_override({"functions.exp_method": 'limit'}):
#     start1 = timeit.default_timer()
#     for _ in range(num_iters):
#         z_enc = x_enc.exp()  # Private
#     print("  Approx:", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
# with cfg.temp_override({"functions.exp_method": 'lut'}):
#     with cfg.temp_override({"functions.exp_all_neg": False}):
#         start1 = timeit.default_timer()
#         for _ in range(num_iters):
#             z_enc = x_enc.exp()  # Private
#         print("  LUT   :", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])


# print("\nLogarithm (LUT faster):")
# print("  Exact :", x.log()[:print_terms])
# with cfg.temp_override({"functions.log_method": 'iter'}):
#     start1 = timeit.default_timer()
#     for _ in range(num_iters):
#         z_enc = x_enc.log()  # Private
#     print("  Approx:", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
# with cfg.temp_override({"functions.log_method": 'lut'}):
#     start1 = timeit.default_timer()
#     for _ in range(num_iters):
#         z_enc = x_enc.log()  # Private
#     print("  LUT   :", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])


# print("\nReciprocal:")
# print("  Exact :", x.reciprocal()[:print_terms])
# with cfg.temp_override({"functions.reciprocal_method": 'NR'}):
#     with cfg.temp_override({"functions.exp_method": 'limit'}):
#         start1 = timeit.default_timer()
#         for _ in range(num_iters):
#             z_enc = x_enc.reciprocal()  # Private
#         print("  Approx:", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
# with cfg.temp_override({"functions.reciprocal_method": 'lut'}):
#     start1 = timeit.default_timer()
#     for _ in range(num_iters):
#         z_enc = x_enc.reciprocal()  # Private
#     print("  LUT   :", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])


# print("\nSquare Root: (LUT faster)")
# print("  Exact :", x.sqrt()[:print_terms])
# with cfg.temp_override({"functions.sqrt_method": 'NR'}):
#     with cfg.temp_override({"functions.inv_sqrt_method": 'NR'}):
#         start1 = timeit.default_timer()
#         for _ in range(num_iters):
#             z_enc = x_enc.sqrt()  # Private
#         print("  Approx:", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
# with cfg.temp_override({"functions.sqrt_method": 'lut'}):
#     with cfg.temp_override({"functions.inv_sqrt_method": 'lut'}):
#         start1 = timeit.default_timer()
#         for _ in range(num_iters):
#             z_enc = x_enc.sqrt()  # Private
#         print("  LUT   :", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])



torch.Size([200])


In [3]:
print("\nSin:")
print("  Exact :", x.sin()[:print_terms])
with cfg.temp_override({"functions.trigonometry_method": 'NR'}):
    start1 = timeit.default_timer()
    for _ in range(num_iters):
        z_enc = x_enc.sin()  # Private
    print("  Approx:", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
    print("  Diff:", (x.sin()-z_enc.get_plain_text()).abs().mean(), (x.sin()-z_enc.get_plain_text()).abs().max())
with cfg.temp_override({"functions.trigonometry_method": 'lut'}):
    start1 = timeit.default_timer()
    for _ in range(num_iters):
        z_enc = x_enc.sin()  # Private
    print("  LUT   :", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
    print("  Diff:", (x.sin()-z_enc.get_plain_text()).abs().mean(), (x.sin()-z_enc.get_plain_text()).abs().max())


print("\nCos:")
print("  Exact :", x.cos()[:print_terms])
with cfg.temp_override({"functions.trigonometry_method": 'NR'}):
    start1 = timeit.default_timer()
    for _ in range(num_iters):
        z_enc = x_enc.cos()  # Private
    print("  Approx:", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
    print("  Diff:", (x.cos()-z_enc.get_plain_text()).abs().mean(), (x.cos()-z_enc.get_plain_text()).abs().max())
with cfg.temp_override({"functions.trigonometry_method": 'lut'}):
    start1 = timeit.default_timer()
    for _ in range(num_iters):
        z_enc = x_enc.cos()  # Private
    print("  LUT   :", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
    print("  Diff:", (x.cos()-z_enc.get_plain_text()).abs().mean(), (x.cos()-z_enc.get_plain_text()).abs().max())


Sin:
  Exact : tensor([ 0.5440,  0.4575,  0.3665,  0.2718,  0.1743,  0.0752, -0.0248, -0.1245])
  Approx: 1 iterations in 0.010527834005188197 sec.: tensor([ 0.5629,  0.4603,  0.3729,  0.2607,  0.1647,  0.0662, -0.0490, -0.1462])
  Diff: tensor(0.0168) tensor(0.0482)
  LUT   : 1 iterations in 0.006757249997463077 sec.: tensor([ 0.5325,  0.4509,  0.3654,  0.2766,  0.1854,  0.0615, -0.0322, -0.1256])
  Diff: tensor(0.0052) tensor(0.0156)

Cos:
  Exact : tensor([-0.8391, -0.8892, -0.9304, -0.9624, -0.9847, -0.9972, -0.9997, -0.9922])
  Approx: 1 iterations in 0.01045999996131286 sec.: tensor([-0.8897, -0.9453, -0.9933, -1.0235, -1.0406, -1.0498, -1.0461, -1.0387])
  Diff: tensor(0.0189) tensor(0.0629)
  LUT   : 1 iterations in 0.005701499991118908 sec.: tensor([-0.8464, -0.8925, -0.9308, -0.9609, -0.9826, -0.9981, -0.9994, -0.9920])
  Diff: tensor(0.0049) tensor(0.0156)


In [4]:
print("\nSigmoid:")
print("  Exact :", x.sigmoid()[:print_terms])
with cfg.temp_override({"functions.sigmoid_tanh_method": 'reciprocal', "functions.reciprocal_method": 'NR', "functions.exp_method": 'limit'}):
    start1 = timeit.default_timer()
    for _ in range(num_iters):
        z_enc = x_enc.sigmoid()  # Private
    print("  Approx:", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
    print("  Diff:", (x.sigmoid()-z_enc.get_plain_text()).abs().mean(), (x.sigmoid()-z_enc.get_plain_text()).abs().max())
with cfg.temp_override({"functions.sigmoid_tanh_method": 'lut'}):
    start1 = timeit.default_timer()
    for _ in range(num_iters):
        z_enc = x_enc.sigmoid()  # Private
    print("  LUT   :", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
    print("  Diff:", (x.sigmoid()-z_enc.get_plain_text()).abs().mean(), (x.sigmoid()-z_enc.get_plain_text()).abs().max())


print("\Tanh:")
print("  Exact :", x.tanh()[:print_terms])
with cfg.temp_override({"functions.sigmoid_tanh_method": 'reciprocal', "functions.reciprocal_method": 'NR', "functions.exp_method": 'limit'}):
    start1 = timeit.default_timer()
    for _ in range(num_iters):
        z_enc = x_enc.tanh()  # Private
    print("  Approx:", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
    print("  Diff:", (x.tanh()-z_enc.get_plain_text()).abs().mean(), (x.tanh()-z_enc.get_plain_text()).abs().max())
with cfg.temp_override({"functions.sigmoid_tanh_method": 'lut'}):
    start1 = timeit.default_timer()
    for _ in range(num_iters):
        z_enc = x_enc.tanh()  # Private
    print("  LUT   :", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
    print("  Diff:", (x.tanh()-z_enc.get_plain_text()).abs().mean(), (x.tanh()-z_enc.get_plain_text()).abs().max())


Sigmoid:
  Exact : tensor([4.5398e-05, 5.0172e-05, 5.5449e-05, 6.1280e-05, 6.7724e-05, 7.4846e-05,
        8.2717e-05, 9.1416e-05])
  Approx: 1 iterations in 0.010023833019658923 sec.: tensor([3.0518e-05, 3.0518e-05, 3.0518e-05, 4.5776e-05, 4.5776e-05, 6.1035e-05,
        6.1035e-05, 7.6294e-05])
  Diff: tensor(0.0004) tensor(0.0020)
  LUT   : 1 iterations in 0.012260959018021822 sec.: tensor([0., 0., 0., 0., 0., 0., 0., 0.])
  Diff: tensor(0.0001) tensor(0.0010)
\Tanh:
  Exact : tensor([-1., -1., -1., -1., -1., -1., -1., -1.])
  Approx: 1 iterations in 0.009671750012785196 sec.: tensor([-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000])
  Diff: tensor(0.0004) tensor(0.0039)
  LUT   : 1 iterations in 0.011867417022585869 sec.: tensor([-1., -1., -1., -1., -1., -1., -1., -1.])
  Diff: tensor(0.0001) tensor(0.0020)


In [5]:
print("\Erf:")
print("  Exact :", x.erf()[:print_terms])
with cfg.temp_override({"functions.erf_method": 'Taylor'}):
    start1 = timeit.default_timer()
    for _ in range(num_iters):
        z_enc = x_enc.erf()  # Private
    print("  Approx:", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
    print("  Diff:", (x.erf()-z_enc.get_plain_text()).abs().mean(), (x.erf()-z_enc.get_plain_text()).abs().max())
with cfg.temp_override({"functions.erf_method": 'lut'}):
    start1 = timeit.default_timer()
    for _ in range(num_iters):
        z_enc = x_enc.erf()  # Private
    print("  LUT   :", num_iters, "iterations in",  timeit.default_timer() - start1, "sec.:", z_enc.get_plain_text()[:print_terms])
    print("  Diff:", (x.erf()-z_enc.get_plain_text()).abs().mean(), (x.erf()-z_enc.get_plain_text()).abs().max())



\Erf:
  Exact : tensor([-1., -1., -1., -1., -1., -1., -1., -1.])
  Approx: 1 iterations in 0.013430875027552247 sec.: tensor([24252978., 33915096., 15197149., 11353300.,  9962572., 11342860.,
        10955472., 24343256.])
  Diff: tensor(4401197.) tensor(33915096.)
  LUT   : 1 iterations in 0.011581791972275823 sec.: tensor([-1., -1., -1., -1., -1., -1., -1., -1.])
  Diff: tensor(0.0002) tensor(0.0044)


#### Advanced mathematics
We are also able to compute more advanced mathematical functions on ```CrypTensors``` using iterative approximations. CrypTen provides MPC support for functions like reciprocal, exponential, logarithm, square root, tanh, etc. Notice that these are subject to numerical error due to the approximations used. 

Additionally, note that some of these functions will fail silently when input values are outside of the range of convergence for the approximations used. These do not produce errors because value are encrypted and cannot be checked without decryption. Exercise caution when using these functions. (It is good practice here to normalize input values for certain models.)